# EDA with awswrangler
- references
    - [aws data warngler](https://velog.io/@hsh/AWSPythonAthena-%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C-%EC%95%84%ED%85%8C%EB%82%98%EC%97%90-%EC%BF%BC%EB%A6%AC%ED%95%98%EA%B8%B0-boto3-vs-pyathena-vs-awswrangler)
    - [Data — AWS Wrangler Query Athena](https://dorian599.medium.com/data-aws-wrangler-query-athena-8be83bc8b091)
    - [sample codes](https://github.com/aws/aws-sdk-pandas/blob/main/tutorials/006%20-%20Amazon%20Athena.ipynb)

### Amazon Athena

[awswrangler](https://github.com/aws/aws-sdk-pandas) has three ways to run queries on Athena and fetch the result as a DataFrame:

- **ctas_approach=True** (Default)

    Wraps the query with a CTAS and then reads the table data as parquet directly from s3.
    
    * `PROS`:
        - Faster for mid and big result sizes.
        - Can handle some level of nested types.
    * `CONS`:
         - Requires create/delete table permissions on Glue.
         - Does not support timestamp with time zone
         - Does not support columns with repeated names.
         - Does not support columns with undefined data types.
         - A temporary table will be created and then deleted immediately.
         - Does not support custom data_source/catalog_id.

- **unload_approach=True and ctas_approach=False**

    Does an UNLOAD query on Athena and parse the Parquet result on s3.

    * `PROS`:
        - Faster for mid and big result sizes.
        - Can handle some level of nested types.
        - Does not modify Glue Data Catalog.
    * `CONS`:
        - Output S3 path must be empty.
        - Does not support timestamp with time zone
        - Does not support columns with repeated names.
        - Does not support columns with undefined data types.

- **ctas_approach=False**

    Does a regular query on Athena and parse the regular CSV result on s3.
    
    * `PROS`:
        - Faster for small result sizes (less latency).
        - Does not require create/delete table permissions on Glue
        - Supports timestamp with time zone.
        - Support custom data_source/catalog_id.
    * `CONS`:
        - Slower (But stills faster than other libraries that uses the regular Athena API)
        - Does not handle nested types at all.

In [100]:
import awswrangler as wr

## Enter your bucket name:

In [152]:
bucket_name = "sm-anomaly-detection"#<your bucket name>
data_path = f"s3://{bucket_name}/data"

## Checking/Creating Glue Catalog Databases

In [153]:
if "awswrangler_test" not in wr.catalog.databases().values:
    wr.catalog.create_database("awswrangler_test")

/tmp/ipykernel_9231/480033085.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if "awswrangler_test" not in wr.catalog.databases().values:


## Creating a Parquet Table from the CSV or parquet files
- **S3에 데이터가 CSV 파일로 존재하는 경우**

In [107]:
import os

In [129]:
#wr.s3.read_csv?

In [120]:
dfs = wr.s3.read_csv(
    path=os.path.join(data_path, "csv"), # folder name
    chunksize=10000, # interable
    dtype_backend="pyarrow"
) 

- **S3에 데이터가 parqeut으로 존재하는 경우**

In [138]:
#wr.s3.read_parquet?

Signature:
wr.s3.read_parquet(
    path: Union[str, List[str]],
    path_root: Optional[str] = None,
    dataset: bool = False,
    path_suffix: Union[str, List[str], NoneType] = None,
    path_ignore_suffix: Union[str, List[str], NoneType] = None,
    ignore_empty: bool = True,
    partition_filter: Optional[Callable[[Dict[str, str]], bool]] = None,
    columns: Optional[List[str]] = None,
    validate_schema: bool = False,
    coerce_int96_timestamp_unit: Optional[str] = None,
    schema: Optional[pyarrow.lib.Schema] = None,
    last_modified_begin: Optional[datetime.datetime] = None,
    last_modified_end: Optional[datetime.datetime] = None,
    version_id: Union[Dict[str, str], str, NoneType] = None,
    dtype_backend: Literal['numpy_nullable', 'pyarrow'] = 'numpy_nullable',
    chunked: Union[bool, int] = False,
    use_threads: Union[bool, int] = True,
    ray_args: Optional[awswrangler.typing.RayReadParquetSettings] = None,
    boto3_session: Optional[boto3.session.Session] = No

In [154]:
dfs = wr.s3.read_parquet(
    path=os.path.join(data_path, "parquet"), # folder name
    chunked=10000, # interable
)
#dfs.sort_values(by="index")

* creating database

In [155]:
for idx, df in enumerate(dfs):
    wr.s3.to_parquet(
        df=df,
        path=os.path.join(data_path, "parquet_from_parqeut"),
        dataset=True,
        mode="append",
        database="awswrangler_test",
        table="reviews"
    )

In [156]:
wr.catalog.table(database="awswrangler_test", table="reviews")

,Column Name,Type,Partition,Comment
0,index,bigint,False,
1,clothing_id,bigint,False,
2,age,bigint,False,
3,title,string,False,
4,review_text,string,False,
5,rating,bigint,False,
6,recommended_ind,bigint,False,
7,positive_feedback_count,bigint,False,
8,division_name,string,False,
9,department_name,string,False,


## Athena query

In [163]:
%%time
quety = """
SELECT division_name, SUM(recommended_ind) AS SUM_CNT
FROM reviews
GROUP BY division_name
"""

wr.athena.read_sql_query(quety, database="awswrangler_test", ctas_approach=False)

CPU times: user 1.01 s, sys: 59.5 ms, total: 1.07 s
Wall time: 3.56 s


,division_name,SUM_CNT
0,General Petite,6707
1,General,11313
2,<NA>,14
3,Initmates,1280


### Reading with ctas_approach=False

In [164]:
%%time
wr.athena.read_sql_query("SELECT * FROM reviews ORDER BY index", database="awswrangler_test", ctas_approach=False)

CPU times: user 1.17 s, sys: 66.9 ms, total: 1.23 s
Wall time: 2.72 s


,index,clothing_id,age,title,review_text,rating,recommended_ind,positive_feedback_count,division_name,department_name,class_name
0,0,767,33,<NA>,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,<NA>,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


### Default with ctas_approach=True - 13x faster (default)

In [158]:
%%time
wr.athena.read_sql_query("SELECT * FROM reviews ORDER BY index", database="awswrangler_test")

CPU times: user 2.42 s, sys: 129 ms, total: 2.55 s
Wall time: 5.96 s


,index,clothing_id,age,title,review_text,rating,recommended_ind,positive_feedback_count,division_name,department_name,class_name
0,22823,829,33,Bumble bees,This is such a cute top. i bought it when it w...,5,1,0,General,Tops,Blouses
1,22824,1020,57,Looooong.,"1. i'm 5'6"" tall with a short waist and long l...",2,1,8,General Petite,Bottoms,Skirts
2,22825,942,51,Nice sweater,I purchased the cedar one. not sure where they...,4,1,0,General,Tops,Sweaters
3,22826,1078,43,Not great,This has a cute aesthetic on the hanger but no...,3,0,6,General Petite,Dresses,Dresses
4,22827,917,43,Slenderizing sweater,I'm 5'1 and a tad busty and this sweater i fou...,5,1,0,General,Tops,Fine gauge
...,...,...,...,...,...,...,...,...,...,...,...
23481,7618,912,40,Too disproportionate,"I love belted sweaters, but this was way too l...",3,0,0,General Petite,Tops,Fine gauge
23482,7619,854,38,Beautiful embroidery,The embroidery around the chest/collar is love...,4,1,0,General Petite,Tops,Knits
23483,7620,868,26,Purchase immediately,I do not think i could express how much i love...,5,1,3,General,Tops,Knits
23484,7621,815,42,Truly outstanding,I reserve my 5 star reviews for products that ...,5,1,25,General,Tops,Blouses


### Using categories to speed up and save memory - 24x faster

In [159]:
%%time
wr.athena.read_sql_query("SELECT * FROM reviews ORDER BY index", database="awswrangler_test", categories=["division_name", "department_name", "class_name"])

CPU times: user 2.47 s, sys: 127 ms, total: 2.6 s
Wall time: 6.05 s


,index,clothing_id,age,title,review_text,rating,recommended_ind,positive_feedback_count,division_name,department_name,class_name
0,22823,829,33,Bumble bees,This is such a cute top. i bought it when it w...,5,1,0,General,Tops,Blouses
1,22824,1020,57,Looooong.,"1. i'm 5'6"" tall with a short waist and long l...",2,1,8,General Petite,Bottoms,Skirts
2,22825,942,51,Nice sweater,I purchased the cedar one. not sure where they...,4,1,0,General,Tops,Sweaters
3,22826,1078,43,Not great,This has a cute aesthetic on the hanger but no...,3,0,6,General Petite,Dresses,Dresses
4,22827,917,43,Slenderizing sweater,I'm 5'1 and a tad busty and this sweater i fou...,5,1,0,General,Tops,Fine gauge
...,...,...,...,...,...,...,...,...,...,...,...
23481,22818,1078,33,"Beautiful, unique dress",I bought this dress on super sale online after...,4,1,1,General Petite,Dresses,Dresses
23482,22819,829,60,"Cute, printed blouse!",This blouse is very cute and will be great for...,4,1,14,General,Tops,Blouses
23483,22820,942,37,Great casual cardigan,"Beautiful cardigan, but it does run large. i'...",5,1,0,General,Tops,Sweaters
23484,22821,829,62,Best if tied at waist,Love the fabric of this button-down. runs real...,4,1,0,General,Tops,Blouses


### Reading with unload_approach=True

In [160]:
%%time
wr.athena.read_sql_query("SELECT * FROM reviews ORDER BY index", database="awswrangler_test", ctas_approach=False, unload_approach=True, s3_output=f"s3://{bucket_name}/data/unload/")

CPU times: user 1.11 s, sys: 57.2 ms, total: 1.17 s
Wall time: 3.58 s


,index,clothing_id,age,title,review_text,rating,recommended_ind,positive_feedback_count,division_name,department_name,class_name
0,0,767,33,<NA>,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,<NA>,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


### Cleaning Up S3

In [165]:
wr.s3.delete_objects(data_path)

### Delete table

In [166]:
wr.catalog.delete_table_if_exists(database="awswrangler_test", table="reviews")

True

### Delete Database

In [167]:
wr.catalog.delete_database('awswrangler_test')